# Sandbox notebook

This notebook contains examples on how to use scikit learn for forecasting.
*Step* is set in the "setup y and X" cell, this lags all the independent X
vars so that we are fiting for / predicting events y *step* time periods away
*y_(t) = X_(t-step)*

## Get started

* File -> Make a copy
* Set the *uname* to your database username
* Run everything

## What score?

"Estimator score method: Estimators have a score method providing a default evaluation criterion for the problem they are designed to solve. This is not discussed on this page, but in each estimator’s documentation."
http://scikit-learn.org/stable/modules/model_evaluation.html

In [2]:
import sys
import os
sys.path.insert(0, "..")

from views_utils import dbutils

import nstep.utils

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
# Utility functions
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Model selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Classifiers
from sklearn import linear_model                        
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier

# Postprocessing
from sklearn.calibration import CalibratedClassifierCV

# Evaluation
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# THIS NEEDS CHANGING
You need to set uname to your database username

In [9]:
VERBOSE = False
CACHE_DATA = False
run_id = "sandbox"
uname = "VIEWSADMIN"

## Database parameters
The default table launched.imp_imp_1 uses imputed y and X.
Many of the derived variables are missing for this table. 
Feel free to use preflight.flight_pgm instead by changing input_schema and input_table.
Remove your locally cahed dataframes in ~/data/df_train.hdf5 for the script to re-download. 

In [5]:
# db parameters
prefix  = "postgresql"
db      = "views"
port    = "5432"
hostname = "VIEWSHOST"

input_schema    = "launched"
input_table     = "imp_imp_1"
output_schema   = "landed_test"
output_table    = run_id

timevar = "month_id"
groupvar = "pg_id"

In [6]:
# Time limits
train_start = 300
train_end = 408
forecast_start = 409
forecast_end = 444

In [7]:
# Varlists

outcomes = ["ged_dummy_sb", "ged_dummy_ns", "ged_dummy_os"]
features = ["bdist3", 
            "ttime_mean", 
            "capdist", 
            "gcp_li_mer", 
            "imr_mean", 
            "mountains_mean", 
            "urban_ih_li",  
            "agri_ih_li", 
            "barren_ih_li",  
            "forest_ih_li",  
            "savanna_ih_li",  
            "shrub_ih_li",  
            "pasture_ih_li"]

ids = [timevar, groupvar]
columns_train = ids + outcomes + features
columns_forecast = ids + outcomes

## Fetch data
This cell fetches data from the database based on the varlists and time limits.
If you change the features or time limits you need to manually delete your local copy of ~/data/df_train for the cell to re-fetch the table from the database. 

In [14]:
# Get data

# try reading cached
try:
    df_train = pd.read_hdf(os.path.expanduser("~/data/df_train.hdf5_"))
    df_test = pd.read_hdf(os.path.expanduser("~/data/df_test.hdf5_")) 
    print("Got df_train and df_test from file")
# get from db if not cached
except:
    print("Didn't find the data on disk, getting from db instead")

    connectstring = dbutils.make_connectstring(prefix, db, uname, hostname, port)

    query_train = dbutils.make_select_limited(
        schema=input_schema, 
        table=input_table, 
        columns=columns_train, 
        timevar=timevar, 
        tmin=train_start, 
        tmax=train_end
        )

    query_forecast = dbutils.make_select_limited(
        schema=input_schema, 
        table=input_table, 
        columns=columns_forecast, 
        timevar=timevar, 
        tmin=forecast_start, 
        tmax=forecast_end
        )
    
    if VERBOSE:
        print("Getting data")
        print(query_train)

    df_train = dbutils.query_to_df(connectstring, query_train, 
        verbose=False, chunksize=100000)
    df_train.set_index(ids, inplace=True)
    df_train.sort_index(inplace=True)
    
    if VERBOSE:
        print(query_test)
    
    df_test = dbutils.query_to_df(connectstring, query_test, verbose=False, chunksize=100000)
    df_test.set_index(ids, inplace=True)

    if CACHE_DATA:
        df_train.to_hdf(os.path.expanduser("~/data/df_train.hdf5"), key='data')
        df_test.to_hdf(os.path.expanduser("~/data/df_test.hdf5"), key='data')

    print("Got df_train and df_test from db")

Didn't find the data on disk, getting from db instead


NameError: name 'query_test' is not defined

# Forecast step
This cell does the lagging of the right handside/X/features

Change the value of *step* to tweak this

In [3]:
# Setup y and X with:
# Shifting X by step to provide a "forecasting" effect
step = 36
# Downsampling 
outcome = "ged_dummy_sb"
share_zeros_keep = 0.1

# y_t, X_(t-step)
y, X = nstep.utils.get_y_X_step(df_train, step, outcome, features, share_zeros_keep)

share_zeros_keep = 1
y_full, X_full = nstep.utils.get_y_X_step(df_train, step, outcome, features, share_zeros_keep)

NameError: name 'df_train' is not defined

# Scikit examples
The following cells show how to fit and predict using scikit. 
Starting with just a classifier we add scaling of inputs, hyperparameter tuning and finally calibration of predict probabilities. 

In [ ]:
def plot_precrec(y_test, y_pred):
    average_precision = average_precision_score(y_test, y_pred)

    print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
    
    precision, recall, _ = precision_recall_curve(y_test, y_pred)

    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
              average_precision))
    plt.show()


In [ ]:
# Single random split train test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Our Multi Layer Perceptron (chosen for fancy name)
mlp =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1, verbose=True)

# Fit our classifier
mlp.fit(X_train, y_train)
# Accuracy score
score = mlp.score(X_test, y_test)
print("Score:", score)

In [ ]:
# Single random split train test with input scaling

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
# Our Multi Layer Perceptron (chosen for fancy name)
mlp =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1, verbose=True)

# Scales an input matrix to mean 0 stdev 1
scaler =  StandardScaler()
# Fit the scaler, this defines the scaling transformation so we can scale X_test the same way
scaler.fit(X_train)
# Apply the scaling transformation to X
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Fit our classifier
mlp.fit(X_train, y_train)
# Accuracy score
score = mlp.score(X_test, y_test)
print("Score:", score)


In [ ]:
# Pipeline example with random split
# notice exact same score as cell above
# pipeline does the fit and transform for us.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

mlp =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1, verbose=True)
scaler =  StandardScaler()

# The pipeline combines the scaler and mlp to a single coherent classifier
clf = make_pipeline(scaler, mlp)

clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print(score)

In [ ]:
# 5-Fold Cross validation score with a pipeline

mlp =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1, verbose=True)
scaler =  StandardScaler()

clf = make_pipeline(scaler, mlp)
cv_scores = cross_val_score(clf, X, y, cv=5)
print("Scores:")
for score in cv_scores:
    print(score)

In [ ]:
# Stratified K-fold, each set gets approx same amount of y=1's

mlp =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1, verbose=True)
scaler =  StandardScaler()

clf = make_pipeline(scaler, mlp)
skf = StratifiedKFold(n_splits = 5)
print("Stratified K-Fold Scores:")
print("k=", )
for train, test in skf.split(X, y):
    clf.fit(X[train], y[train])
    score = clf.score(X[test], y[test])
    print(score)

In [ ]:
# Exhaustive grid search for hyperparameter tuning
# This loops over all the combinations of parameters and performs K-fold cross validation
# The best parameters are found and printed

mlp =  MLPClassifier(random_state=1)
scaler =  StandardScaler()

# Syntax is pipelinecomponent__parameter, notice double underscores
# Parameters to try are given as tuples
parameters = {
    'mlp__hidden_layer_sizes' : ((5,5), (10,5), (5,5,5)),
    'mlp__solver' : ('lbfgs', 'sgd', 'adam'),
    'mlp__alpha' : (1e-3, 1e-5, 1e-7)
}

pipeline = Pipeline([
    ('scaler', scaler),
    ('mlp', mlp)
])

print("pipeline:", [name for name, _ in pipeline.steps])

K = 3
cpu_cores = 2

grid_search = GridSearchCV(pipeline, parameters, verbose=1, cv=K, n_jobs=cpu_cores)
grid_search.fit(X, y)

print("Best score: %0.5f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
y_pred = grid_search.predict(X_full)
plot_precrec(y_full, y_pred)

In [ ]:
# Exhaustive grid search for hyperparameter tuning, Random Forest Example
# This loops over all the combinations of parameters and performs K-fold cross validation

rf =  RandomForestClassifier()
scaler =  StandardScaler()

# Syntax is pipelinecomponent__parameter, notice double underscores
parameters = {
    'rf__n_estimators' : (5, 15, 35)
}

pipeline = Pipeline([
    ('scaler', scaler),
    ('rf', rf)
])

print("pipeline:", [name for name, _ in pipeline.steps])

K = 3
cpu_cores = 2

grid_search = GridSearchCV(pipeline, parameters, verbose=1, cv=K, n_jobs=cpu_cores)
grid_search.fit(X, y)
y_pred = grid_search.predict(X_full)

print("Best score: %0.5f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

plot_precrec(y_full, y_pred)

In [ ]:
# Gridsearch with StratifiedKFold on  K neighbors classifier
# Principle Component Analysis (PCA) is included, are all the features useful?

scaler =  StandardScaler()
pca = PCA()
knc =  KNeighborsClassifier()
K_splits = 3
skf = StratifiedKFold(n_splits = K_splits)

# Syntax is pipelinecomponent__parameter, notice double underscores
parameters = {
    'knc__n_neighbors' : (1,3,5,15,30),
    'pca__n_components' : (1,2,3,4,5,6,7,8)
}

pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('knc', knc)
])

print("pipeline:", [name for name, _ in pipeline.steps])

cpu_cores = 2

grid_search = GridSearchCV(pipeline, parameters, verbose=1, cv=skf, n_jobs=cpu_cores)
grid_search.fit(X, y)

clf_best = grid_search.best_estimator_
df_gs_resuls = pd.DataFrame(grid_search.cv_results_)
print("Best score: %0.5f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print(df_gs_results)

## Calibration

Scikit has a method for calibrating classifiers called CalibratedClassifierCV, which uses a sigmoid.

Another option is doing it manually, by fitting a logistic regression of the predicted probabilities from the uncalibrated model on the actual outcomes. 

Note that I'm not using a separate calibration set, just the non-downsampledtrainingdata, so don't follow my example!

In [ ]:
# First a pipeline with a StandardScaler and an MLPClassifier are trained on downsampled data (X).
# Then a CalibratedClassifierCV (cc) object based on the pipeline is fited on the full training data.

# These are downsampled
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Train clf on downsampled
mlp =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1, verbose=True)
scaler =  StandardScaler()
clf = make_pipeline(scaler, mlp)
clf.fit(X_train, y_train)

# Calibrate the classifier on full data
cc = CalibratedClassifierCV(clf, cv='prefit')
cc.fit(X_full, y_full)


# predict probabilities and calibrated probabilities from full data
p_y = clf.predict_proba(X_full)
p_y_calib = cc.predict_proba(X_full)


# predict_proba gives one columns for each class in our data, 
# for our binary case we have two classes, y=0 and y=1. 
# p_y[:,0] gives probs of y=0
# p_y[:,1] gives probs of y=1
# p_1 is prob zero, pc_1 is calibrated prob zero
df_p = pd.DataFrame({
    'p_0' : p_y[:,0],
    'p_1' : p_y[:,1],
    'pc_0' : p_y_calib[:,0],
    'pc_1' : p_y_calib[:,1]})
print("Pay attention to the axes limits!")
df_p.plot(kind='scatter', x='p_0', y='pc_0', title="Raw vs calibrated p(y=0)")
df_p.plot(kind='scatter', x='p_1', y='pc_1', title="Raw vs calibrated p(y=1)")
plt.show()

In [ ]:
# First a pipeline with a StandardScaler and an MLPClassifier are trained on downsampled data (X).
# Then a CalibratedClassifierCV (cc) object based on the pipeline is fited on the full training data.
# The cc uses a sigmoid (logit-style) by default

# These are downsampled
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Train clf on downsampled
mlp =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 5), random_state=1, verbose=True)
scaler =  StandardScaler()
clf = make_pipeline(scaler, mlp)
clf.fit(X_train, y_train)

# predict probabilities on full data
p_y = clf.predict_proba(X_full)

# Train a logit model 
# y_actual = alpha + beta * p_y 
# where p_y is uncalibrated probs
logit = LogisticRegression()
logit.fit(p_y[:,1].reshape(-1, 1), y_full)
p_y_calib = logit.predict_proba(p_y[:,1].reshape(-1, 1))

# p_1 is prob zero, pc_1 is calibrated prob zero
df_p = pd.DataFrame({
    'p_0' : p_y[:,0],
    'p_1' : p_y[:,1],
    'pc_0' : p_y_calib[:,0],
    'pc_1' : p_y_calib[:,1]})
print("Pay attention to the axes limits!")
df_p.plot(kind='scatter', x='p_0', y='pc_0', title="Raw vs calibrated p(y=0)")
df_p.plot(kind='scatter', x='p_1', y='pc_1', title="Raw vs calibrated p(y=1)")
plt.show()

df_test = pd.DataFrame({
    'actual' : y_full,
    'p' : p_y[:,1],
    'pc' : p_y_calib[:,1]
})

In [ ]:
df_test['error_raw'] = df_test['actual'] - df_test['p']
df_test['error_calib'] = df_test['actual'] - df_test['pc']

plt.figure()
plt.title("raw error")
df_test['error_raw'].hist(bins=100)
plt.show()
plt.figure()
plt.title("calibrated error")
df_test['error_calib'].hist(bins=100)
plt.show()
print(df_test.describe())

avg_precision_raw = average_precision_score(df_test['actual'], df_test['p'])
avg_precision_calib = average_precision_score(df_test['actual'], df_test['pc'])

print("precision raw:  ", avg_precision_raw)
print("precision calib:", avg_precision_calib)

# Statsmodels
The cells below show some statsmodels examples

Note that THESE ARE NOT FORECASTING! 

Models are y_t = beta*X_t

Todo:
In the get_y_X_step() function I should add an option for getting lagged groups also

## Random effects

In [ ]:
# THIS IS PREDICTING y_t with X_t NOT FORECASTING!
# Random effects
df_re = df_train.sample(n=1000000)
formula_sm = "ged_dummy_sb ~ bdist3 + ttime_mean + capdist + gcp_li_mer + imr_mean + mountains_mean + urban_ih_li + agri_ih_li + barren_ih_li + forest_ih_li + savanna_ih_li + shrub_ih_li + pasture_ih_li"
# get_level_values(1) gives the values of pg_id because df has had set_index(['month_id', 'pg_id'])
md = smf.mixedlm(formula_sm, df_re, groups=df_re.index.get_level_values(1))
mdf = md.fit()
print(mdf.summary())
df_re['pred_y'] = mdf.predict()
df_re['error'] = df_re['pred_y'] - df_re['ged_dummy_sb']
df_re[['ged_dummy_sb', 'pred_y', 'error']].describe()